In [32]:
ecommerce_data = [
    {
        "user_id": "U001", "name": "Alice", "age": 25, "city": "Paris",
        "orders": [
            {"order_id": "O1", "date": "2024-01", "items": [("laptop", 999, 1), ("mouse", 25, 2)], "status": "delivered"},
            {"order_id": "O2", "date": "2024-02", "items": [("keyboard", 75, 1)], "status": "pending"},
            {"order_id": "O3", "date": "2024-03", "items": [("monitor", 300, 1), ("webcam", 50, 1)], "status": "delivered"},
            {"order_id": "O4", "date": "2024-04", "items": [("monitor", 300, 1), ("pad", 15, 1)], "status": "delivered"}
        ]
    },
    {
        "user_id": "U002", "name": "Bob", "age": 35, "city": "Lyon", 
        "orders": [
            {"order_id": "O4", "date": "2024-01", "items": [("laptop", 999, 1)], "status": "delivered"},
            {"order_id": "O5", "date": "2024-02", "items": [("mouse", 25, 1), ("pad", 15, 3)], "status": "delivered"}
        ]
    },
    {
        "user_id": "U003", "name": "Charlie", "age": 28, "city": "Paris",
        "orders": [
            {"order_id": "O6", "date": "2024-01", "items": [("keyboard", 75, 2), ("mouse", 25, 1)], "status": "cancelled"},
            {"order_id": "O7", "date": "2024-03", "items": [("laptop", 999, 1), ("monitor", 300, 2)], "status": "delivered"}
        ]
    },
    {
        "user_id": "U004", "name": "Diana", "age": 30, "city": "Paris",
        "orders": [
            {"order_id": "O8", "date": "2024-03", "items": [("mouse", 25, 1), ("monitor", 300, 1)], "status": "delivered"},
        ]
    },
    {
        "user_id": "U005", "name": "Eve", "age": 40, "city": "Lyon",
        "orders": [
            {"order_id": "O9", "date": "2024-03", "items": [("laptop", 999, 1), ("keyboard", 75, 1)], "status": "delivered"},
        ]
    }
]

Extrayez tous les produits achetés par des utilisateurs parisiens avec quantité > 1

In [14]:
prdt_user_paris = [(data['name'], item[0], item[2]) for data in ecommerce_data 
                   for order in data['orders'] for item in order['items']
                   if data['city'] == 'Paris' and item[2] > 1 and order["status"] == "delivered"]

In [15]:
prdt_user_paris

[('Alice', 'mouse', 2), ('Charlie', 'monitor', 2)]

Calculez le "score de fidélité" par utilisateur: nombre_commandes * 10 + total_dépensé * 0.1, mais seulement pour les utilisateurs avec au moins 2 commandes livrées.

In [12]:
score_fide = [
    (
        data["name"],
        sum(1 for order in data["orders"] if order["status"] == "delivered") * 10 + sum(
            sum(price * quantity for _, price, quantity in order["items"])
            for order in data["orders"] if order["status"] == "delivered"
        ) * 0.1
    )
    for data in ecommerce_data
    if sum(1 for order in data["orders"] if order["status"] == "delivered") >= 2
]

In [13]:
score_fide

[('Alice', 201.4), ('Bob', 126.9)]

Trouvez les utilisateurs qui ont commandé le même produit dans différentes commandes (détection de récurrence d'achat).

In [42]:
users_with_recurrent = {}
for user in ecommerce_data:
    user_id = user["user_id"]
    name = user["name"]
    product_orders = {}  # Dictionnaire pour compter dans combien de commandes un produit apparaît
    
    # Parcourir les commandes de l'utilisateur
    for order in user["orders"]:  # Corrigé : user["orders"] au lieu de orders
        # Parcourir les items de chaque commande
        for item in order["items"]:
            produit = item[0]  # Nom du produit
            product_orders[produit] = product_orders.get(produit, 0) + 1
    
    # Vérifier les produits qui apparaissent dans plusieurs commandes
    recurrent_products = [produit for produit, count in product_orders.items() if count > 1]
    if recurrent_products:  # Si des produits sont récurrents
        users_with_recurrent[user_id] = {"name": name, "products": recurrent_products}

# Afficher les résultats
for user_id, info in users_with_recurrent.items():
    print(f"Utilisateur {info['name']} a acheté les produits suivants dans plusieurs commandes :")
    for produit in info["products"]:
        print(f"  - {produit} {count}")

Utilisateur Alice a acheté les produits suivants dans plusieurs commandes :
  - monitor 2


In [27]:
# Étape 1 : Identifier tous les produits uniques
all_products = set()
for user in ecommerce_data:
    for order in user['orders']:
        for item in order['items']:
            all_products.add(item[0])
all_products = sorted(list(all_products))  # Trier pour un ordre cohérent

# Étape 2 : Créer la matrice
prdt_user = []
for user in ecommerce_data:
    # Dictionnaire pour compter les quantités par produit pour cet utilisateur
    quantities = {product: 0 for product in all_products}
    for order in user['orders']:
        for item in order['items']:
            product, _, quantity = item
            quantities[product] += quantity
    # Convertir en liste alignée sur all_products
    prdt_user.append([quantities[product] for product in all_products])

# Afficher la matrice avec les noms des utilisateurs et produits
print("Produits :", all_products)
for i, user in enumerate(ecommerce_data):
    print(f"{user['name']} : {prdt_user[i]}")

Produits : ['keyboard', 'laptop', 'monitor', 'mouse', 'pad', 'webcam']
Alice : [1, 1, 2, 2, 1, 1]
Bob : [0, 1, 0, 1, 3, 0]
Charlie : [2, 1, 2, 1, 0, 0]


In [2]:
# Dataset réseau social
social_network = [
    {"user": "Alice", "posts": [{"content": "Hello world", "likes": 15, "tags": ["tech", "python"]}, 
                                {"content": "Data science rocks", "likes": 23, "tags": ["data", "ml"]}],
     "following": ["Bob", "Diana"], "followers": ["Bob", "Charlie", "Eve"]},
    {"user": "Bob", "posts": [{"content": "Morning coffee", "likes": 8, "tags": ["life"]},
                              {"content": "Python tutorial", "likes": 45, "tags": ["tech", "python", "tutorial"]}],
     "following": ["Alice", "Charlie"], "followers": ["Alice", "Frank"]},
    {"user": "Charlie", "posts": [{"content": "Weekend vibes", "likes": 12, "tags": ["life", "weekend"]}],
     "following": ["Alice"], "followers": ["Alice", "Bob"]}
]

In [39]:
# Initialiser le dictionnaire des connexions mutuelles
mutual_connections = {}

# Parcourir chaque utilisateur A
for user_A in social_network:
    mutual_list = []  # Liste pour stocker les connexions mutuelles de A
    user_A_name = user_A["user"]  # Nom de l'utilisateur A
    following = user_A["following"]  # Personnes suivies par A
    
    # Parcourir chaque personne B suivie par A
    for person_B in following:
        # Trouver l'utilisateur B dans social_network
        for user_B in social_network:
            if user_B["user"] == person_B:
                # Vérifier si A est dans les followers de B
                if user_A_name in user_B["followers"]:
                    mutual_list.append(person_B)
                break  # Sortir une fois B trouvé
    
    # Ajouter l'entrée au dictionnaire
    mutual_connections[user_A_name] = mutual_list

# Afficher le résultat
print(mutual_connections)

{'Alice': ['Bob'], 'Bob': ['Alice', 'Charlie'], 'Charlie': ['Alice']}


In [84]:
from collections import defaultdict

# Étape 1 : Extraire tous les (tag, likes)
list_tags = []
total_likes = 0
total_posts = 0

for user in social_network:
    for post in user['posts']:
        tags = post['tags']
        likes = post['likes']

        total_likes += likes
        total_posts += 1

        for tag in tags:
            list_tags.append((tag, likes))

# Étape 2 : Regrouper les likes par tag
tag_likes = defaultdict(list)

for tag, likes in list_tags:
    tag_likes[tag].append(likes)

# Étape 3 : Moyenne globale
average_likes_all = total_likes / total_posts if total_posts else 0

# Étape 4 : Calcul du taux d'engagement par tag
engagement_by_tag = {}

for tag, likes_list in tag_likes.items():
    avg_likes_tag = sum(likes_list) / len(likes_list)
    engagement_rate = (avg_likes_tag / average_likes_all) * 100 if average_likes_all else 0
    engagement_by_tag[tag] = engagement_rate

# Étape 5 : Affichage
for tag, rate in sorted(engagement_by_tag.items(), key=lambda x: x[1], reverse=True):
    print(f"Tag: {tag:15} | Taux d'engagement: {rate:.2f}%")


Tag: tutorial        | Taux d'engagement: 218.45%
Tag: tech            | Taux d'engagement: 145.63%
Tag: python          | Taux d'engagement: 145.63%
Tag: data            | Taux d'engagement: 111.65%
Tag: ml              | Taux d'engagement: 111.65%
Tag: weekend         | Taux d'engagement: 58.25%
Tag: life            | Taux d'engagement: 48.54%


In [80]:
from collections import defaultdict

customer_produit = defaultdict(set)

for user in ecommerce_data:
    customer = user['name']
    for order in user['orders']:
        for item, _, _ in order['items']:
            customer_produit[customer].add(item)

# Affichage :
for customer, produits in customer_produit.items():
    print(f"{customer} → {produits}")


Alice → {'monitor', 'webcam', 'laptop', 'mouse', 'pad', 'keyboard'}
Bob → {'pad', 'laptop', 'mouse'}
Charlie → {'monitor', 'keyboard', 'laptop', 'mouse'}


In [79]:
from itertools import combinations

# Étape 1 : construire client → set(produits)
client_produits = {}

for user in ecommerce_data:
    client = user['name']
    produits = set()
    for order in user['orders']:
        for item, _, _ in order['items']:
            produits.add(item)
    client_produits[client] = produits

# Étape 2 : comparer chaque paire de clients
clients_en_commun = []

for client1, client2 in combinations(client_produits.keys(), 2):
    produits1 = client_produits[client1]
    produits2 = client_produits[client2]
    if produits1 & produits2:  # intersection non vide
        clients_en_commun.append((client1, client2))

# Affichage
print("Clients ayant acheté des produits en commun :")
for c1, c2 in clients_en_commun:
    print(f" - {c1} & {c2}")


Clients ayant acheté des produits en commun :
 - Alice & Bob
 - Alice & Charlie
 - Bob & Charlie


Pour chaque utilisateur, trouvez les produits achetés par des utilisateurs similaires (même ville) qu'il n'a pas encore achetés.


In [3]:
ecommerce_data

[{'user_id': 'U001',
  'name': 'Alice',
  'age': 25,
  'city': 'Paris',
  'orders': [{'order_id': 'O1',
    'date': '2024-01',
    'items': [('laptop', 999, 1), ('mouse', 25, 2)],
    'status': 'delivered'},
   {'order_id': 'O2',
    'date': '2024-02',
    'items': [('keyboard', 75, 1)],
    'status': 'pending'},
   {'order_id': 'O3',
    'date': '2024-03',
    'items': [('monitor', 300, 1), ('webcam', 50, 1)],
    'status': 'delivered'},
   {'order_id': 'O4',
    'date': '2024-04',
    'items': [('monitor', 300, 1), ('pad', 15, 1)],
    'status': 'delivered'}]},
 {'user_id': 'U002',
  'name': 'Bob',
  'age': 35,
  'city': 'Lyon',
  'orders': [{'order_id': 'O4',
    'date': '2024-01',
    'items': [('laptop', 999, 1)],
    'status': 'delivered'},
   {'order_id': 'O5',
    'date': '2024-02',
    'items': [('mouse', 25, 1), ('pad', 15, 3)],
    'status': 'delivered'}]},
 {'user_id': 'U003',
  'name': 'Charlie',
  'age': 28,
  'city': 'Paris',
  'orders': [{'order_id': 'O6',
    'date': '

In [45]:
cust={}
for user in ecommerce_data:
    client=user['name']
    ville=user['city']
    produits=set()
    for order in user['orders']:
        for item, _, _ in order['items']:
            produits.add(item)
        cust[client]=produits
        

In [46]:
cust

{'Alice': {'keyboard', 'laptop', 'monitor', 'mouse', 'pad', 'webcam'},
 'Bob': {'laptop', 'mouse', 'pad'},
 'Charlie': {'keyboard', 'laptop', 'monitor', 'mouse'}}

In [7]:
cust=[(user['name'],user['city'],item) for user in ecommerce_data for order in user['orders'] 
      for item,montant,quanity in order['items']]

In [4]:
cust=set(cust)

In [38]:
from collections import defaultdict

# Étape 1 : construire les structures
produits_par_utilisateur = {}
ville_par_utilisateur = {}

for user in ecommerce_data:
    nom = user['name']
    ville = user['city']
    ville_par_utilisateur[nom] = ville

    produits = set()
    for order in user['orders']:
        if order['status'] == 'delivered':
            for item in order['items']:
                produits.add(item[0])
    produits_par_utilisateur[nom] = produits
print(produits_par_utilisateur)
print(ville_par_utilisateur)
# Étape 2 : suggestions avec sources
suggestions = {}

for utilisateur in produits_par_utilisateur:
    ville = ville_par_utilisateur[utilisateur]
    produits_utilisateur = produits_par_utilisateur[utilisateur]

    produits_autres = defaultdict(set)  # produit : {autres utilisateurs}
    for autre in produits_par_utilisateur:
        if autre != utilisateur and ville_par_utilisateur[autre] == ville:
            for produit in produits_par_utilisateur[autre]:
                if produit not in produits_utilisateur:
                    produits_autres[produit].add(autre)

    suggestions[utilisateur] = produits_autres

# Étape 3 : affichage
for utilisateur, produits_sources in suggestions.items():
    ville = ville_par_utilisateur[utilisateur]
    print(f"\n🔎 {utilisateur} pourrait découvrir ces produits livrés dans sa ville ({ville}) :")

    if produits_sources:
        for produit, acheteurs in sorted(produits_sources.items()):
            acheteurs_str = ", ".join(sorted(acheteurs))
            print(f"   - {produit} (acheté par : {acheteurs_str})")
    else:
        print("   ✅ Aucun nouveau produit à suggérer, il/elle a déjà tout acheté !")


{'Alice': {'monitor', 'webcam', 'mouse', 'laptop', 'pad'}, 'Bob': {'laptop', 'pad', 'mouse'}, 'Charlie': {'laptop', 'monitor'}, 'Diana': {'monitor', 'mouse'}, 'Eve': {'laptop', 'keyboard'}}
{'Alice': 'Paris', 'Bob': 'Lyon', 'Charlie': 'Paris', 'Diana': 'Paris', 'Eve': 'Lyon'}

🔎 Alice pourrait découvrir ces produits livrés dans sa ville (Paris) :
   ✅ Aucun nouveau produit à suggérer, il/elle a déjà tout acheté !

🔎 Bob pourrait découvrir ces produits livrés dans sa ville (Lyon) :
   - keyboard (acheté par : Eve)

🔎 Charlie pourrait découvrir ces produits livrés dans sa ville (Paris) :
   - mouse (acheté par : Alice, Diana)
   - pad (acheté par : Alice)
   - webcam (acheté par : Alice)

🔎 Diana pourrait découvrir ces produits livrés dans sa ville (Paris) :
   - laptop (acheté par : Alice, Charlie)
   - pad (acheté par : Alice)
   - webcam (acheté par : Alice)

🔎 Eve pourrait découvrir ces produits livrés dans sa ville (Lyon) :
   - mouse (acheté par : Bob)
   - pad (acheté par : Bob)
